# Coursera Capstone

This notebook will be mainly used for the capstone project

### Week 1 Assignment

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


### Week 3 Assignment

#### Instructions

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

#### Steps

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1. Start by creating a new Notebook for this assignment.


2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

![Table](https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/7JXaz3NNEeiMwApe4i-fLg_40e690ae0e927abda2d4bde7d94ed133_Screen-Shot-2018-06-18-at-7.17.57-PM.png?expiry=1589068800000&hmac=qzYkoLq5XsqE06VeplzJ13UfdrokXnyfhItpfpO3vps)

3. To create the above dataframe:

    * The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
    * Only process the cells that have an assigned borough. Ignore cells with a borough that is __Not assigned__.
    * More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that __M5A__ is listed twice and has two neighborhoods: __Harbourfront__ and __Regent Park__. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in __row 11__ in the above table.
    * If a cell has a borough but a __Not assigned__ neighborhood, then the neighborhood will be the same as the borough.
    * Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
    * In the last cell of your notebook, use the __.shape__ method to print the number of rows of your dataframe.


4. Submit a link to your Notebook on your Github repository. (__10 marks__)

__Note__: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

The package is so popular that there is a plethora of tutorials and examples on how to use it. Here is a very good Youtube video on how to use the BeautifulSoup package: https://www.youtube.com/watch?v=ng2o98k983k

Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

#### My code

__*1. Using BeautifulSoup to get the table*__

In [3]:
# !conda install -c conda-forge bs4 --yes
from bs4 import BeautifulSoup
import urllib.request

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#request = urllib.request.Request(url, headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36'})
#response = urllib.request.urlopen(request, timeout=200)
  
response = urllib.request.urlopen(url, timeout=200)
html = response.read()
response.close()

soup = BeautifulSoup(html, 'html.parser')

In [4]:
# Save a version to avoid the potential network errors

with open ("wiki.txt", "w") as f:
    for line in soup:
        f.write(str(line))
print("Done!")

Done!


In [5]:
pc_ls = []
br_ls = []
nh_ls = []

ls = soup.tbody.get_text().split('\n\n')
    
for i,item in enumerate(ls): 
    item = item.strip()
    if i % 3 == 0: pc_ls.append(item)
    if i % 3 == 1: br_ls.append(item)
    if i % 3 == 2: nh_ls.append(item)

df1 = pd.DataFrame([pc_ls,br_ls,nh_ls]).T
df1.columns = df1.iloc[0,:]
df1 = df1[1:]
df1.reset_index(inplace=True, drop=True)
df1.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


__*2. Using Pandas to get the table*__

In [6]:
# !conda install -c conda-forge lxml --yes
df2 = pd.read_html(url)[0]
df2.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


__*3. Process the table*__

We use df2 from part2 for further process.

In [7]:
# Remove lines without Borough values
df = df2[df2['Borough']!='Not assigned']
df.reset_index(inplace=True, drop=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
# Find lines that share the same postal code
df_gp = df.groupby("Postal Code").count()
df_gp[df_gp['Borough']>2]

,Borough,Neighborhood
Postal Code,,


The result indicates the postal codes are already unique in the table.

In [9]:
# Replace NaN in Neighborhood with Borough value
df[df['Neighborhood'].isna()]

,Postal Code,Borough,Neighborhood


The result indicates no such case exists.

In [10]:
df.shape

(103, 3)

----